# Chassis proto

In [ ]:
import time
import gym
import nle

import numpy as np
import matplotlib.pyplot as plt

We hide the NLE under several layers of wrappers. From the core to the shell:
1. `ReplayToFile` saves the seeds and the takes actions into a file for later inspection and replay.
2. `NLEAtoN` maps ascii actions to opaque actions accpeted by the NLE.
3. `NLEObservationPatches` patches tty-screens, botched by the cr-lf misconfiguration of the NLE's tty term emulator and NetHacks displays (lf only).
4. `NLEFeatureExtractor` adds extra features generated on-the-fly from the current NLE's observation. 

In [ ]:
from nle_toolbox.utils.replay import ReplayToFile
from nle_toolbox.utils.env.wrappers import NLEObservationPatches, NLEAtoN, NLEFeatureExtractor


def factory():
    return \
    NLEFeatureExtractor(
        NLEObservationPatches(
            NLEAtoN(
                ReplayToFile(
                    gym.make(
                        'NetHackChallenge-v0'
                    ),
                    save_on='done',
                    sticky=True,
                    folder='./replays',
                )
            )
        ),
        k=2,  # vicinity radius
    )

## Basic GUI Handling

NetHack's gui is not as intricate as in some other games. We need to deal
with menus, text prompts, messages and y/n questions. In order to analyze
the interface details and player's journey through the UI, we first implement
a simple command evaluator.

In [ ]:
from collections import deque

def gui_run(
    env,
    *commands
):
    pipe0 = deque([])
    obs = env.reset()
    for cmd in commands:
        pipe0.extend(cmd)
        while pipe0:
            obs, *ignore = env.step(pipe0.popleft())

        yield obs

### Menus

There are two types of menus on NetHack: single paged and multipage. Single
page menus popup in the middle of the terminal ontop of the dungeon map (and
are sort of `dirty`, meaning that they have arbitrary symbols around them),
while multi-page menus take up the entire screen after clearing it. Overlaid
menu regions appear to be right justified, while their contents' text is
left-justified. All menus are modal, i.e. capture the keyboard input until
exited. Some menus are static, i.e. display some information, while other
are interactive, i.e. allow item selection with letters or punctuation. However,
both kinds share two special control keys. The space `\0x20` (`\040`, 32,
`<SPACE>`) advances to the next page, or closes the menu, if the page was
the last or the only one. The escape `\0x1b` (`\033`, 27, `^[`) immediately
exits any menu.

In [ ]:
import re

rx_menu_is_overlay = re.compile(
    r"""
    \(
        (
            # either we get a short single-page overlay menu
            end
        |
            # or a long multi-page menu
            (?P<cur>\d+)
            \s+ of \s+
            (?P<tot>\d+)
        )
    \)\s*$
    """,
    re.VERBOSE | re.IGNORECASE | re.MULTILINE | re.ASCII,
)

rx_menu_item = re.compile(
    r"""
    ^(
        (?P<letter>[a-z])
        \s+[\-\+]
    )?\s+
    (?P<item>.*)
    $
    """,
    re.VERBOSE | re.IGNORECASE | re.ASCII,
)

The following detects the type of the menu (overlay/fullscreen), its number
of pages, and extracts its content.

In [ ]:
from collections import namedtuple
GUIRawMenu = namedtuple('GUIRawMenu', 'n_pages,n_page,is_overlay,data')

def menu_extract(lines):
    col, row, match = 80, 0, None

    # detect menu box
    matches = map(rx_menu_is_overlay.search, lines)
    for rr, m in enumerate(matches):
        if m is None:
            continue

        beg, end = m.span()
        if beg <= col:
            col, row, match = beg, rr, m

    # extract the menu and the pagination
    if match is None:
        return None

    is_overlay = False
    content = tuple([ll[col:].rstrip() for ll in lines[:row]])
    n_page, n_pages = match.group('cur', 'tot')
    if n_pages is None:
        n_page, n_pages, is_overlay = 1, 1, True

    return GUIRawMenu(
        int(n_pages),
        int(n_page),
        is_overlay,
        content,
    )

The following function extracts raw data from a menu and enumerates all
items, which can be interacted with.

In [ ]:
GUIMenu = namedtuple('GUIMenu', 'n_pages_left,items,letters')

def menu_parse(obs):
    tty_lines = obs['tty_chars'].view('S80')[:, 0]

    # Assume a menu is on the screen. Detect which one (single,
    # multi), (letters if interactive) and extract its content.
    menu = menu_extract([ll.decode('ascii') for ll in tty_lines])
    if menu is None:
        return None

    # extract menu items
    items, letters = [], {}
    for entry in menu.data:
        m = rx_menu_item.match(entry)
        if m is not None:
            lt, it = m.group('letter', 'item')
            items.append(it)
            if lt is not None:
                letters[lt] = it

    # return the parsed menu
    return GUIMenu(
        # number of additional pages
        menu.n_pages - menu.n_page,
        # the line-by-line content of the menu
        items,
        # which items can be interacted with
        letters,
    )

## Top Line Messages

The game reports events, displays status or information in the top two lines
of the screen. The NLE also provides the raw data in the `message` field of
the observation. When NetHack generally announces in the top line, however,
if it wants to communicate a single message longer than `80` characters, the
game allows it to spill over to the second line, appending a `--More--` suffix
to it. The game does the same if it has several short messages to announce.
In both cases NetHack's gui expects the user to confirm or dismiss each message
by pressing Space, Enter or Escape.

Some helper functions to fetch and detect multi-part messages.

In [ ]:
def fetch_message(obs, *, top=False):
    if top:
        # padded with whitespace on the right
        message = bytes(obs['tty_chars'][:2])
    else:
        # has trailing zero bytes
        message = bytes(obs['message'].view('S256')[0])

    return message.rstrip().decode('ascii')

def has_more_messages(obs):
    # get the top line from tty-chars
    # XXX `Misc(*obs['misc']).xwaitingforspace` reacts to menus as well,
    #  bu we want pure multi-part messages.
    return '--More--' in fetch_message(obs, top=True)

<br>

## Putting it all together

Below is a wrapper, which handles menus (unless an interaction is required) and
fetches all consecutive messages.

In [ ]:
from gym import Wrapper

class Chassis(Wrapper):
    """Handle multi-part messages, yes-no-s, and other gui events, which
    were not deliberately requested by downstream policies.
    """
    def __init__(self, env, *, top=False):
        super().__init__(env)
        self.top = top

    def reset(self):
        obs, rew, done, info = self.fetch(self.env.reset(), 0., False, None)
        return obs

    def step(self, action):
        return self.fetch(*self.env.step(action))

    def fetch(self, *tx):
        # first we detect and parse menus, since messages cannot
        # appear when they are active
        tx = self.fetch_menus(*tx)
        tx = self.fetch_messages(*tx)
        return tx

    def fetch_messages(self, obs, rew=0., done=False, info=None):
        buffer = []
        while has_more_messages(obs) and not done:
            # inside this loop the message CANNOT be empty by design
            buffer.append(fetch_message(obs, top=self.top))
            obs, rew, done, info = self.env.step(' ')  # send SPACE

        # the final message may be empty so we сheck for it
        message = fetch_message(obs, top=self.top)
        if message:
            buffer.append(message)
        self.messages = tuple(buffer)

        # XXX obs['message'] contains the last message
        return obs, rew, done, info

    def fetch_menus(self, obs, rew=0., done=False, info=None):
        """Handle single and multi-page interactive and static menus.
        """
        page = menu_parse(obs)
        if page is not None:
            # parse menus and collect all their data unless interactive
            pages = []
            while not page.letters and page.n_pages_left > 0:
                pages.append(page)
                obs, rew, done, info = self.env.step(' ')  # send SPACE
                page = menu_parse(obs)

            pages.append(page)
            if not page.letters:
                obs, rew, done, info = self.env.step(' ')  # send SPACE

            # join the pages collected so far
            self.menu = GUIMenu(
                page.n_pages_left,
                tuple([it for page in pages for it in page.items]),
                page.letters,
            )

        else:
            self.menu = None

        # XXX we'd better listen to special character action when
        #  dealing with interactive menus.
        return obs, rew, done, info

<br>

## Testing

Let's test it in bulk.

In [ ]:
seed = 12513325507677477210, 18325590921330101247  # multi
# seed = 1251332550767747710, 18325590921330101247  # single
seed = 125133255076774710, 18325590921330101247  # single
# seed = 13765371332493407478, 12246923801353953927
seed = 12604736832047991440, 12469632217503715839
# seed = None


with Chassis(factory(), top=False) as env:
    seed = env.seed(seed)

    menus = []
    for obs in gui_run(
        env,
        '',
#         '\033;j.',
#         '\033;h.',
        '\033;lllll.',
#         '\033#help\015j ',
#         '\033#help\015j  ',
#         '\033i',
#         '\033#enhance\015',
#         '\033e*\033',
        '\033D,\015',
    ):
        menus.append(menu_parse(obs))

In [ ]:
seed

In [ ]:
env.messages

In [ ]:
env.menu.items if env.menu is not None else None

In [ ]:
env.menu.letters if env.menu is not None else None

In [ ]:
obs['tty_chars'].view('S80')[:, 0]

In [ ]:
assert False